In [1]:
import datetime
import gspread
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from urllib.request import urlopen
import time
from time import sleep
from basketball_reference_scraper.pbp import get_pbp

pd.options.mode.chained_assignment = None  # default='warn'


In [2]:
nba_schedule_df = pd.read_csv('22_23_nba_schedule.csv')
nba_schedule_df['DATE'] = pd.to_datetime(nba_schedule_df['DATE'], format='%b %d %Y')

In [3]:
# nba_schedule_df_retro = nba_schedule_df[
#     (nba_schedule_df.DATE>='2022-11-18') &
#     (nba_schedule_df.DATE<=yesterday)
# ].reset_index(drop=True)

In [4]:
nba_schedule_df_retro = nba_schedule_df[
    (nba_schedule_df.DATE>='2023-04-02') &
    (nba_schedule_df.DATE<='2023-04-09')
].reset_index(drop=True)

In [5]:
# test = get_pbp('2022-11-18', 'IND', 'HOU') # 'IND' 'HOU'
# # test = get_pbp('2022-11-18', 'ORL', 'CHI')
# test['TIME_REMAINING'] = test.TIME_REMAINING.replace('24:00.0','12:00.0')
# test['TIME_REMAINING'] = pd.to_datetime(test['TIME_REMAINING'])

In [6]:
nba_wp_results = pd.DataFrame()

for game_number in range(len(nba_schedule_df)):
    if (game_number % 9) == 0:
        sleep(120)
    date = nba_schedule_df_retro.loc[game_number, "DATE"]
    away_team = nba_schedule_df_retro.loc[game_number, "AWAY_TEAM"]
    home_team = nba_schedule_df_retro.loc[game_number, "HOME_TEAM"]
    pbp_df = get_pbp(date, away_team, home_team)
    display(pbp_df)
    break
    print(str(date) + '-' + str(game_number) + '-' + away_team + '-' + home_team)
    wp_results = calculate_game_wp(pbp_df, date, away_team, home_team)
    nba_wp_results = pd.concat([nba_wp_results, wp_results]).reset_index(drop=True)

/Users/michaelmarzec/opt/anaconda3/lib/python3.9/site-packages/basketball_reference_scraper/pbp.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(str(table))[0]


,QUARTER,TIME_REMAINING,TORONTO_ACTION,CHARLOTTE_ACTION,TORONTO_SCORE,CHARLOTTE_SCORE


In [23]:
nba_wp_results.to_csv('round_7.csv')

In [ ]:
##### daily runs

In [ ]:
# simulating a scheduled run
yesterday = pd.Timestamp.today().normalize() - pd.DateOffset(days=1)
yesterday_games_df = nba_schedule_df[nba_schedule_df.DATE == yesterday].reset_index(drop=True)
yesterday_number_of_games = len(yesterday_games_df) 

In [ ]:
for game_number in range(yesterday_number_of_games):
    date = yesterday_games_df.loc[game_number, "DATE"]
    away_team = yesterday_games_df.loc[game_number, "AWAY_TEAM"]
    home_team = yesterday_games_df.loc[game_number, "HOME_TEAM"]
    pbp_df = get_pbp(date, away_team, home_team)
    wp_results = calculate_game_wp(pbp_df, date, away_team, home_team)
    nba_wp_results = pd.concat([nba_wp_results, wp_results]).reset_index(drop=True)

In [ ]:
# date = '2022-11-19'
# away_team = 'PHI'
# home_team = 'MIN'


In [12]:
def calculate_game_wp(pbp_df, date, away_team, home_team):
    pbp_df = pbp_df.rename(columns={
        pbp_df.columns[4]:"AWAY_SCORE",
        pbp_df.columns[5]:"HOME_SCORE"
    })

    pbp_df = pbp_df[['QUARTER','TIME_REMAINING','AWAY_SCORE','HOME_SCORE']]
    pbp_df['TIME_REMAINING'] = pbp_df.TIME_REMAINING.replace('24:00.0','12:00.0')
    pbp_df['TIME_REMAINING'] = pd.to_datetime(pbp_df['TIME_REMAINING'])
    pbp_df['TIME_REMAINING'] = pd.to_datetime(pbp_df['TIME_REMAINING'],format= '%H:%M:%S').dt.time
    pbp_df['AWAY_SCORE'] = pbp_df['AWAY_SCORE'].astype(int)
    pbp_df['HOME_SCORE'] = pbp_df['HOME_SCORE'].astype(int)

    quarter_beginning = '12:00:00'
    quarter_ending = '00:00:00'
    overtime_beginning = '05:00:00'

    list_of_quarters = list(pbp_df['QUARTER'].unique())

    full_df = pd.DataFrame()
    for quarter in list_of_quarters:
        quarter_df = pbp_df[pbp_df['QUARTER'] == quarter]

        if quarter == 1:# add beginning of 1st quarter
            quarter_df.loc[-1] = [1, quarter_beginning, 0, 0] 
            quarter_df.index = quarter_df.index + 1  
            quarter_df = quarter_df.sort_index()

            end_row = {
                'QUARTER': [quarter],
                'TIME_REMAINING': quarter_ending,
                'AWAY_SCORE': [quarter_df.iloc[-1,2]],
                'HOME_SCORE': [quarter_df.iloc[-1,3]]
            }

            end_row = pd.DataFrame(data=end_row)
            quarter_df = pd.concat([quarter_df, end_row], axis=0).reset_index(drop=True)

            temp_date = str(datetime.datetime.strptime('1900-01-01', '%Y-%m-%d').date())
            quarter_df['TIME_REMAINING'] = pd.to_datetime(temp_date + " " + quarter_df.TIME_REMAINING.astype(str))
            quarter_df['TIME_ELAPSED'] = (quarter_df['TIME_REMAINING'] - quarter_df['TIME_REMAINING'].shift(-1))
            quarter_df['INDICATOR'] = np.where(
                quarter_df['QUARTER'] == quarter_df['QUARTER'].shift(-1),
                ((quarter_df['TIME_REMAINING'] - quarter_df['TIME_REMAINING'].shift(-1))/60000000000).view(int),
                0
            )

            full_df = pd.concat([full_df, quarter_df], axis=0)


        elif quarter in (2,3,4):
            start_row = end_row
            start_row['QUARTER'] = quarter
            start_row['TIME_REMAINING'] = quarter_beginning

            quarter_df = quarter_df.reset_index(drop=True)
            quarter_df.index += 1

            quarter_df = pd.concat([start_row, quarter_df],axis=0)
            quarter_df = quarter_df.sort_index()

            end_row = {
                'QUARTER': [quarter],
                'TIME_REMAINING': quarter_ending,
                'AWAY_SCORE': [quarter_df.iloc[-1,2]],
                'HOME_SCORE': [quarter_df.iloc[-1,3]]
            }

            end_row = pd.DataFrame(data=end_row)
            quarter_df = pd.concat([quarter_df, end_row], axis=0).reset_index(drop=True)

            temp_date = str(datetime.datetime.strptime('1900-01-01', '%Y-%m-%d').date())
            quarter_df['TIME_REMAINING'] = pd.to_datetime(temp_date + " " + quarter_df.TIME_REMAINING.astype(str))
            quarter_df['TIME_ELAPSED'] = (quarter_df['TIME_REMAINING'] - quarter_df['TIME_REMAINING'].shift(-1))
            quarter_df['INDICATOR'] = np.where(
                quarter_df['QUARTER'] == quarter_df['QUARTER'].shift(-1),
                ((quarter_df['TIME_REMAINING'] - quarter_df['TIME_REMAINING'].shift(-1))/60000000000).view(int),
                0
            )

            full_df = pd.concat([full_df, quarter_df], axis=0)

        else:
            start_row = end_row
            start_row['QUARTER'] = quarter
            start_row['TIME_REMAINING'] = overtime_beginning

            quarter_df = quarter_df.reset_index(drop=True)
            quarter_df.index += 1

            quarter_df = pd.concat([start_row, quarter_df],axis=0)
            quarter_df = quarter_df.sort_index()

            end_row = {
                'QUARTER': [quarter],
                'TIME_REMAINING': quarter_ending,
                'AWAY_SCORE': [quarter_df.iloc[-1,2]],
                'HOME_SCORE': [quarter_df.iloc[-1,3]]
            }

            end_row = pd.DataFrame(data=end_row)
            quarter_df = pd.concat([quarter_df, end_row], axis=0).reset_index(drop=True)

            temp_date = str(datetime.datetime.strptime('1900-01-01', '%Y-%m-%d').date())
            quarter_df['TIME_REMAINING'] = pd.to_datetime(temp_date + " " + quarter_df.TIME_REMAINING.astype(str))
            quarter_df['TIME_ELAPSED'] = (quarter_df['TIME_REMAINING'] - quarter_df['TIME_REMAINING'].shift(-1))
            quarter_df['INDICATOR'] = np.where(
                quarter_df['QUARTER'] == quarter_df['QUARTER'].shift(-1),
                ((quarter_df['TIME_REMAINING'] - quarter_df['TIME_REMAINING'].shift(-1))/60000000000).view(int),
                0
            )

            full_df = pd.concat([full_df, quarter_df], axis=0)

    full_df['LEADER'] = np.where(
        full_df.HOME_SCORE > full_df.AWAY_SCORE,
        'HOME_TEAM',
        'AWAY_TEAM',
    )

    full_df['LEADER'] = np.where(
        full_df.HOME_SCORE == full_df.AWAY_SCORE,
        'NEITHER',
        full_df.LEADER
    )
    full_df_gb = full_df.groupby('LEADER')['INDICATOR'].sum().reset_index() 

    teams = ['AWAY_TEAM','HOME_TEAM']
    total_seconds = full_df_gb.INDICATOR.sum()

    results_row = pd.DataFrame(columns=['DATE','AWAY_TEAM','HOME_TEAM','AWAY_WP','HOME_WP'],index=[0])
    results_row['DATE'] = date
    results_row['AWAY_TEAM'] = away_team
    results_row['HOME_TEAM'] = home_team
    
    try:
        away_seconds_leading = full_df_gb[full_df_gb.LEADER == 'AWAY_TEAM'].INDICATOR.values[0]
    except:
        away_seconds_leading = 0
    away_wp = away_seconds_leading / total_seconds

    try:
        home_seconds_leading = full_df_gb[full_df_gb.LEADER == 'HOME_TEAM'].INDICATOR.values[0]
    except:
        home_seconds_leading = 0
    home_wp = home_seconds_leading / total_seconds
    
    results_row['AWAY_WP'] = away_wp
    results_row['HOME_WP'] = home_wp

    return results_row
